## Classification de chiffres manuscrits avec un réseau de neurones profond

Ce notebook détaille la construction, l'entraînement et l'évaluation d'un réseau de neurones profond pour la classification d'images de chiffres manuscrits. Le dataset utilisé est le célèbre MNIST, souvent considéré comme le "Hello World" de l'apprentissage profond.

### Contenu :
1. **Préparation des données** : Chargement du dataset MNIST, mise à l'échelle des images, division en ensembles d'entraînement, de validation et de test.
2. **Construction du modèle** : Création d'un réseau de neurones avec deux couches cachées.
3. **Entraînement du modèle** : Utilisation de l'optimiseur Adam et de la fonction de perte `sparse_categorical_crossentropy`.
4. **Évaluation du modèle** : Mesure de la perte et de la précision sur l'ensemble de test.


-------

## Importer les packages	

In [7]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

# ces ensembles de données seront stockés dans C:\Users\*NOM_UTILISATEUR*\tensorflow_datasets\...



## Chargement et prétaitement des données

In [8]:
# tfds.load charge un ensemble de données (ou le télécharge puis le charge si c'est la première fois) 
# dans notre cas, nous sommes intéressés par le MNIST; le nom de l'ensemble de données est le seul argument obligatoire
# il y a d'autres arguments que nous pouvons spécifier, qui peuvent nous être utiles
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# with_info=True nous fournira également un tuple contenant des informations sur la version, les caractéristiques, le nombre d'échantillons
# nous utiliserons ces informations un peu plus bas et nous les stockerons dans mnist_info

# as_supervised=True chargera l'ensemble de données dans une structure à 2 tuples (entrée, cible) 
# sinon, as_supervised=False renverrait un dictionnaire
# bien sûr, nous préférons avoir nos entrées et cibles séparées 

# une fois que nous avons chargé l'ensemble de données, nous pouvons facilement extraire l'ensemble d'entraînement et de test avec les références intégrées
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# par défaut, TF a des ensembles d'entraînement et de test, mais pas d'ensembles de validation
# nous devons donc le diviser nous-mêmes

# nous commençons par définir le nombre d'échantillons de validation en tant que % des échantillons d'entraînement
# c'est aussi là que nous utilisons mnist_info (nous n'avons pas besoin de compter les observations)
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
# convertissons ce nombre en entier, car un float pourrait causer une erreur en cours de route
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# stockons également le nombre d'échantillons de test dans une variable dédiée (au lieu d'utiliser celle de mnist_info)
num_test_samples = mnist_info.splits['test'].num_examples
# encore une fois, nous préférerions un entier (plutôt que le float par défaut)
num_test_samples = tf.cast(num_test_samples, tf.int64)

# nous aimerions mettre à l'échelle nos données pour rendre le résultat plus numériquement stable
# dans ce cas, nous préférerons simplement avoir des entrées entre 0 et 1
# définissons une fonction appelée : scale, qui prendra une image MNIST et son label
def scale(image, label):
    # nous nous assurons que la valeur est un float
    image = tf.cast(image, tf.float32)
    # comme les valeurs possibles pour les entrées sont de 0 à 255 (256 nuances différentes de gris)
    # si nous divisons chaque élément par 255, nous obtiendrons le résultat souhaité -> tous les éléments seront entre 0 et 1 
    image /= 255.

    return image, label

# la méthode .map() nous permet d'appliquer une transformation à un ensemble de données
# nous avons déjà décidé que nous obtiendrions les données de validation à partir de mnist_train, donc 
scaled_train_and_validation_data = mnist_train.map(scale)

# enfin, nous mettons à l'échelle et regroupons les données de test
# nous le mettons à l'échelle pour qu'il ait la même amplitude que l'entraînement et la validation
# il n'est pas nécessaire de le mélanger, car nous ne nous entraînerons pas sur les données de test
# il y aurait un seul batch, égal à la taille des données de test
test_data = mnist_test.map(scale)

# mélangeons également les données

BUFFER_SIZE = 10000
# ce paramètre BUFFER_SIZE est ici pour les cas où nous traitons d'énormes ensembles de données
# alors nous ne pouvons pas mélanger tout l'ensemble de données en une seule fois car nous ne pouvons pas tout mettre en mémoire
# donc TF ne stocke que "BUFFER_SIZE" échantillons en mémoire à la fois et les mélange
# si BUFFER_SIZE=1 => aucun mélange ne se produira réellement
# si BUFFER_SIZE >= num échantillons => le mélange est uniforme
# BUFFER_SIZE entre les deux - une optimisation computationnelle pour approximer un mélange uniforme

# heureusement pour nous, il existe une méthode de mélange prête à l'emploi et nous devons simplement spécifier la taille du buffer
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# une fois que nous avons mis à l'échelle et mélangé les données, nous pouvons procéder à l'extraction réelle de l'entraînement et de la validation
# nos données de validation seront égales à 10% de l'ensemble d'entraînement, que nous avons déjà calculé
# nous utilisons la méthode .take() pour prendre autant d'échantillons
# enfin, nous créons un lot avec une taille de lot égale au nombre total d'échantillons de validation
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

# de même, les train_data sont tout le reste, donc nous sautons autant d'échantillons qu'il y en a dans l'ensemble de validation
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# déterminons la taille du batch
BATCH_SIZE = 100

# nous pouvons également profiter de l'occasion pour regrouper les données d'entraînement
# cela serait très utile lorsque nous nous entraînons, car nous pourrions itérer sur les différents lots
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

# regroupez les données de test
test_data = test_data.batch(num_test_samples)

# prend le prochain batch (c'est le seul batch)
# car as_supervized=True, nous avons une structure à 2 tuples
validation_inputs, validation_targets = next(iter(validation_data))


## Construction du modèle

In [9]:
input_size = 784
output_size = 10
# Utilise la même taille de couche cachée pour les deux couches cachées. Ce n'est pas une nécessité.
hidden_layer_size = 50
    
# définir la structure du modèle
model = tf.keras.Sequential([
    
    # la première couche (la couche d'entrée)
    # chaque observation est de 28x28x1 pixels, donc c'est un tenseur de rang 3
    # il y a une méthode pratique 'Flatten' qui prend simplement notre tenseur 28x28x1 et le transforme en un vecteur (None,) 
    # ou (28x28x1,) = (784,)
    # cela nous permet de créer un réseau neuronal feedforward
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # couche d'entrée
    
    # tf.keras.layers.Dense implémente essentiellement : output = activation(dot(input, weight) + bias)
    # il prend plusieurs arguments, mais les plus importants pour nous sont hidden_layer_size et la fonction d'activation
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1ère couche cachée
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2ème couche cachée
    
    # la couche finale n'est pas différente, nous nous assurons simplement de l'activer avec softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # couche de sortie
])


## Choix de l'optimiseur et de la fonction de perte

In [10]:
# nous définissons l'optimiseur que nous souhaitons utiliser, 
# la fonction de perte,
# et les métriques qui nous intéressent à chaque itération
# optimizer='adam' : L'optimiseur "Adam" est une méthode de descente de gradient stochastique qui est basée sur une estimation adaptative des moments de premier et second ordre.
# loss='sparse_categorical_crossentropy' : Cette fonction de perte est utilisée pour les problèmes de classification où les classes sont mutuellement exclusives (c'est-à-dire, chaque entrée appartient exactement à une catégorie). "Sparse" signifie que nous utilisons une seule étiquette entière pour chaque observation pour représenter les classes (par exemple, [2, 3, 1, ...]) plutôt que des vecteurs one-hot.
# metrics=['accuracy'] : La métrique "accuracy" (précision) calcule la proportion d'entrées correctement classées par le modèle.
    
    
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


## Entraînement du modèle

In [11]:
# déterminer le nombre maximal d'époques
NUM_EPOCHS = 5

# nous entraînons le modèle en spécifiant :
# les données d'entraînement
# le nombre total d'époques
# et les données de validation que nous avons créées nous-mêmes au format : (entrées, cibles)
    # train_data : Les données sur lesquelles le modèle sera entraîné.
    # epochs=NUM_EPOCHS : Le nombre d'époques est le nombre de fois que le modèle passera par l'ensemble de données d'entraînement. Dans ce cas, il passera 5 fois.
    # validation_data=(validation_inputs, validation_targets) : Les données de validation sont utilisées pour évaluer la performance du modèle après chaque époque. Cela donne une indication de la manière dont le modèle se généralise sur des données qu'il n'a jamais vues auparavant.
    # verbose=2 : Ce paramètre contrôle la quantité d'informations à afficher pendant l'entraînement. Avec verbose=2, l'output affichera une ligne par époque.

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)


Epoch 1/5


540/540 - 3s - loss: 0.4157 - accuracy: 0.8849 - val_loss: 0.2099 - val_accuracy: 0.9410 - 3s/epoch - 6ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1782 - accuracy: 0.9477 - val_loss: 0.1515 - val_accuracy: 0.9560 - 2s/epoch - 3ms/step
Epoch 3/5
540/540 - 2s - loss: 0.1358 - accuracy: 0.9610 - val_loss: 0.1240 - val_accuracy: 0.9628 - 2s/epoch - 3ms/step
Epoch 4/5
540/540 - 2s - loss: 0.1122 - accuracy: 0.9667 - val_loss: 0.1063 - val_accuracy: 0.9695 - 2s/epoch - 3ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0944 - accuracy: 0.9717 - val_loss: 0.0918 - val_accuracy: 0.9748 - 2s/epoch - 3ms/step


## Test du modèle

Après avoir été entraîné sur les données d'entraînement et validé sur les données de validation, nous testons la puissance de prédiction finale de notre modèle en l'exécutant sur le jeu de données de test que l'algorithme n'a JAMAIS vu auparavant.

Il est très important de réaliser que la manipulation des hyperparamètres sur-entraine le jeu de données de validation.

Le test est l'instance finale absolue. On ne doit pas tester avant d'avoir complètement ajusté votre modèle.

Si On ajuste notre modèle après le test, nous commencerons à sur-entrainer le jeu de données de test, ce qui en annulera l'objectif.

In [12]:
test_loss, test_accuracy = model.evaluate(test_data)
# model.evaluate(test_data) : Cette fonction évalue la performance du modèle sur les données de test. Elle renvoie la perte (loss) et la précision (accuracy) du modèle sur ces données.
# print('Perte lors du test : {0:.2f}. Précision lors du test : {1:.2f}%'.format(test_loss, test_accuracy*100.)) : Cette ligne affiche la perte et la précision du modèle sur les données de test. Le formatage {0:.2f} et {1:.2f} est utilisé pour afficher les nombres avec deux décimales.



print('Perte lors du test : {0:.2f}. Précision lors du test : {1:.2f}%'.format(test_loss, test_accuracy*100.))


1/1 [==============================] - 0s 282ms/step - loss: 0.1124 - accuracy: 0.9675
Perte lors du test : 0.11. Précision lors du test : 96.75%


En utilisant le modèle initial et les hyperparamètres donnés dans ce notebook, la précision finale du test devrait être d'environ 97%.

Chaque fois que le code est exécuté à nouveau, nous obtenons une précision différente car les lots sont mélangés, les poids sont initialisés différemment, etc.